## Setup

In [ ]:
import pandas as pd
import numpy as np
from sensorfabric.athena import athena
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import utils

In [155]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pyathena

In [ ]:
import sys
!{sys.executable} -m pip install -U kaleido


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 15.9 MB/s eta 0:00:0000:0100:01


In [ ]:
%set_env AWS_PROFILE=opensci

env: AWS_PROFILE=opensci


In [ ]:
db = athena(database='menolife', workgroup='amyris', offlineCache=True)

# Pulled in query below to test # Users w at least a day of data and meeting premenopausal criteria. This says 20,567. No Way That's Right

In [ ]:
premenopausalusers_query = """
SELECT DISTINCT o.healthid
FROM onboardingpp o
INNER JOIN symptoms s ON o.healthid = s.uid
INNER JOIN (
    SELECT uid
    FROM symptoms_sum
    WHERE days_symptoms_recorded > 0
) ss ON o.healthid = ss.uid
WHERE (o.questions = 'Let''s customize your personal health tracker!'
        OR o.questions = 'Choose the symptoms you deal with the most.'
        OR o.questions = 'How would you describe your periods?'
        OR o.questions = 'Are you on birth control?'
        OR o.questions = 'How did you enter menopause?')
    AND LENGTH(o.answers) > 0
    AND (o.answers NOT LIKE '%Hot Flashes/Night Sweats%'
        AND o.answers NOT LIKE '%I don''t have a period anymore%'
        AND o.answers NOT LIKE '%IUD%'
        AND o.answers NOT LIKE '%Pill%'
        AND o.answers NOT LIKE '%Implant%'
        AND o.answers NOT LIKE '%Shot%'
        AND o.answers NOT LIKE '%Vaginal Ring%'
        AND o.answers NOT LIKE '%Patch%'
        AND o.answers NOT LIKE '%Naturally%'
        AND o.answers NOT LIKE '%Medically%'
        AND o.answers NOT LIKE '%Hysterectomy%'
        AND o.answers NOT LIKE '%Chemotherapy%'
        OR o.answers = 'Unpredictable'
        OR o.answers = 'Regular cycle'
        OR o.answers = 'none'
    );
"""

symptoms_query = """
SELECT s.uid, s.name, COUNT(s.name) AS count
FROM symptoms s
INNER JOIN (
    SELECT uid
    FROM symptoms_sum
    WHERE days_symptoms_recorded > 0
) ss ON s.uid = ss.uid
INNER JOIN onboardingpp o ON s.uid = o.healthid
GROUP BY s.uid, s.name
ORDER BY s.uid, COUNT(s.name) DESC;
"""

premenopausalusers = db.execQuery(premenopausalusers_query, cached=True)
symptoms_data = db.execQuery(symptoms_query, cached=True)

print(premenopausalusers.shape)



(20567, 1)


In [ ]:
premenopausalusers.head()

symptoms_data.shape

(168995, 3)

### Premenopausal query. Make sure we're only keeping 1 uid per person: 1680 results seems more like it.

In [ ]:
premenopausalusers_query = """
WITH query1 AS (
    SELECT DISTINCT healthid
    FROM onboardingpp o
    WHERE (questions = 'Let''s customize your personal health tracker!'
        OR questions = 'Choose the symptoms you deal with the most.'
        AND questions = 'How would you describe your periods?'
        AND questions = 'Are you on birth control?'
        AND questions = 'How did you enter menopause?')
        AND length(answers) > 0
        AND (
            answers NOT LIKE '%Hot Flashes/Night Sweats%'
            AND answers NOT LIKE '%I don''t have a period anymore%'
            AND answers NOT LIKE '%IUD%'
            AND answers NOT LIKE '%Pill%'
            AND answers NOT LIKE '%Implant%'
            AND answers NOT LIKE '%Shot%'
            AND answers NOT LIKE '%Vaginal Ring%'
            AND answers NOT LIKE '%Patch%'
            AND answers NOT LIKE '%Naturally%'
            AND answers NOT LIKE '%Medically%'
            AND answers NOT LIKE '%Hysterectomy%'
            AND answers NOT LIKE '%Chemotherapy%'
            OR answers = 'Unpredictable'
            OR answers = 'Regular cycle'
            OR answers = 'none'
        )
),
query2 AS (
    SELECT DISTINCT s.uid, s.name
    FROM symptoms s
    INNER JOIN (
        SELECT DISTINCT uid
        FROM symptoms_sum
        WHERE days_symptoms_recorded > 0
    ) ss ON s.uid = ss.uid
)
SELECT q1.healthid, MIN(q2.uid) AS uid, MIN(q2.name) AS name
FROM query1 q1
INNER JOIN query2 q2 ON q1.healthid = q2.uid
GROUP BY q1.healthid
ORDER BY uid;
"""

premenopausalusers = db.execQuery(premenopausalusers_query, cached=True)
print(premenopausalusers.shape)
premenopausalusers.head()


(1680, 3)


,healthid,uid,name
0,5f931b94a42cd9d7053ec20f,5f931b94a42cd9d7053ec20f,spotting
1,5f935278a42cd9d7053ec212,5f935278a42cd9d7053ec212,bloating
2,5f935f5fa42cd9d7053ec21b,5f935f5fa42cd9d7053ec21b,cramps
3,5f937679a42cd9d7053ec227,5f937679a42cd9d7053ec227,anxiety
4,5f944dbfa42cd9d7053ec25d,5f944dbfa42cd9d7053ec25d,bloating


### Compare to query of just UIDS from onboarding, which should be a BIGGER not a smaller number. The Query below returns 3419 users. So we need the intersection to be < 3419. Check

In [ ]:
premenopausalusers=db.execQuery("""
SELECT distinct healthid
FROM onboardingpp o
WHERE (questions = 'Let''s customize your personal health tracker!'
    OR questions = 'Choose the symptoms you deal with the most.'
    AND   questions = 'How would you describe your periods?'
    AND questions = 'Are you on birth control?'
    AND questions = 'How did you enter menopause?')
    AND length(answers) > 0
    AND (
        (
        answers NOT LIKE '%Hot Flashes/Night Sweats%'
        and answers not like '%have a period anymore%'
        and answers not like '%IUD%'
        and answers not like '%Pill%'
        and answers not like '%Implant%'
        and answers not like '%Shot%'
        and answers not like '%Vaginal Ring%'
        and answers not like '%Patch%'
        and answers not like '%Naturally%'
        and answers not like '%Medically%'
        and answers not like '%Hysterectomy%'
        and answers not like '%Chemotherapy%'
        )
        OR answers = 'Unpredictable'
        OR answers = 'Regular cycle'
        OR answers ='none')
        order by healthid
        """,cached=True)
print(premenopausalusers.shape)

(3419, 1)


In [ ]:
premenopausalusers.head()

,healthid
0,5f931b94a42cd9d7053ec20f
1,5f935278a42cd9d7053ec212
2,5f935f5fa42cd9d7053ec21b
3,5f937679a42cd9d7053ec227
4,5f944dbfa42cd9d7053ec25d


In [ ]:
users = db.execQuery("""
                    select s.uid, s.name, count(s.name) count from symptoms s 
                    inner join 
                    (select uid from symptoms_sum
                    where days_symptoms_recorded >= 0) ss
                    on s.uid = ss.uid
                    group by s.uid, s.name
                    order by s.uid, count(s.name) desc
                    """, cached=True)
print(users.shape)

(186139, 3)


## Lets modify this to give me a table of symptoms for all the ids that meet these criteria

In [ ]:
premenopausalusers_symptoms_query= """
WITH query1 AS (
    SELECT DISTINCT healthid
    FROM onboardingpp 
    WHERE (questions = 'Let''s customize your personal health tracker!'
        OR questions = 'Choose the symptoms you deal with the most.'
        AND questions = 'How would you describe your periods?'
        AND questions = 'Are you on birth control?'
        AND questions = 'How did you enter menopause?')
        AND length(answers) > 0
        AND (
            (
            answers NOT LIKE '%Hot Flashes/Night Sweats%'
            AND answers NOT LIKE '%have a period anymore%'
            AND answers NOT LIKE '%IUD%'
            AND answers NOT LIKE '%Pill%'
            AND answers NOT LIKE '%Implant%'
            AND answers NOT LIKE '%Shot%'
            AND answers NOT LIKE '%Vaginal Ring%'
            AND answers NOT LIKE '%Patch%'
            AND answers NOT LIKE '%Naturally%'
            AND answers NOT LIKE '%Medically%'
            AND answers NOT LIKE '%Hysterectomy%'
            AND answers NOT LIKE '%Chemotherapy%'
            )
            OR answers = 'Unpredictable'
            OR answers = 'Regular cycle'
            OR answers = 'none'
        )
),
query2 AS (
    SELECT s.uid, s.name, COUNT(s.name) AS count 
    FROM symptoms s
    INNER JOIN query1 q1 ON s.uid = q1.healthid
    INNER JOIN (
        SELECT uid
        FROM symptoms_sum ss
        WHERE days_symptoms_recorded > 0
        AND ss.uid IN (SELECT distinct healthid FROM query1)
    ) ss ON q1.healthid = ss.uid
    GROUP BY s.uid, s.name
)
SELECT q2.uid, q2.name, q2.count
FROM query2 q2
ORDER BY q2.uid, q2.count DESC;
"""

premenopausalusers_symptoms = db.execQuery(premenopausalusers_symptoms_query, cached=True)
print(premenopausalusers_symptoms.shape)
premenopausalusers_symptoms.head()


(8414, 3)


,uid,name,count
0,5f931b94a42cd9d7053ec20f,spotting,1
1,5f935278a42cd9d7053ec212,cramps,4
2,5f935278a42cd9d7053ec212,yeast_infection,3
3,5f935278a42cd9d7053ec212,bloating,1
4,5f935f5fa42cd9d7053ec21b,mood_swings,3


In [ ]:
#Is this still keeping only the correct uids? 

# Calculate the unique number of UIDs
unique_uids_count = len(premenopausalusers_symptoms['uid'].unique())

# Display the unique number of UIDs
print("Unique number of UIDs:", unique_uids_count)

Unique number of UIDs: 1680


In [ ]:
#Lets see if the df looks right

frame = pd.DataFrame()  # Empty dataframe that will contain all the dataset.
uidmap = {}
# Start breaking the users frame and extract data from it to add to the data matrix.
temp = None  # Temp dictionary that holds the symptom counts for each user.
for uid, sname, scount in zip(premenopausalusers_symptoms['uid'].values, premenopausalusers_symptoms['name'].values, premenopausalusers_symptoms['count'].values):
    if not (uid in uidmap):
        # If temp is not None (everything except for the first uid then we will concat the last uid to the frame.
        if not (temp is None):
            temp = pd.Series(temp)
            temp = temp.to_frame().T
            frame = pd.concat([frame, temp], ignore_index=True)
        temp = {}  # Start fresh again.
        temp['uid'] = uid
        uidmap[uid] = 1
    temp[sname] = scount
frame = frame.set_index(['uid']) # Set the 'uid' as the index, so it becomes easier to query.
frame = frame.fillna(0)

In [ ]:
frame.head(5)

,spotting,cramps,yeast_infection,bloating,mood_swings,joint_pain,fatigue,anxiety,irregular_beat,headaches,...,hair_loss,itchy_skin,vertigo,vaginal_dryness,tingling_extremeties,vomiting,ovulation,painful_intercourse,uti,nipple_discharge
uid,,,,,,,,,,,,,,,,,,,,,
5f931b94a42cd9d7053ec20f,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5f935278a42cd9d7053ec212,0,4,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5f935f5fa42cd9d7053ec21b,0,1,0,0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5f937679a42cd9d7053ec227,0,0,0,1,0,1,3,2,1,1,...,0,0,0,0,0,0,0,0,0,0
5f944dbfa42cd9d7053ec25d,0,0,0,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Note some users are still reporting hot flashes. Remove them. 

frame = frame[(frame['hot_flashes'] == 0) & (frame['night_sweats'] == 0)]

In [ ]:
frame.head()

,spotting,cramps,yeast_infection,bloating,mood_swings,joint_pain,fatigue,anxiety,irregular_beat,headaches,...,hair_loss,itchy_skin,vertigo,vaginal_dryness,tingling_extremeties,vomiting,ovulation,painful_intercourse,uti,nipple_discharge
uid,,,,,,,,,,,,,,,,,,,,,
5f931b94a42cd9d7053ec20f,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5f935278a42cd9d7053ec212,0,4,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5f935f5fa42cd9d7053ec21b,0,1,0,0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5f937679a42cd9d7053ec227,0,0,0,1,0,1,3,2,1,1,...,0,0,0,0,0,0,0,0,0,0
5f944dbfa42cd9d7053ec25d,0,0,0,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Now how many do I have left?

unique_uids = frame.index.unique().tolist()
unique_uids_size = len(unique_uids)
print("Size of unique_uids:", unique_uids_size)



Size of unique_uids: 1408


In [ ]:
frame.to_csv('premenopausalsymptoms.csv')

## Moving on to the perimenopausal UID set

In [ ]:
## Perimenopausal Query

## QUERY: How would you describe yoru periods?

In [ ]:
query = """
select * from onboardingpp 
  where (questions = 'How would you describe your periods?')
  and length(answers) > 0
"""

results = db.execQuery(query, cached=True)

In [ ]:
results.head(60)

,healthid,community_id,questions,answers
0,60f3a934bea29a1eb66cefea,60f3a93227df7d3103e102bf,How would you describe your periods?,Regular cycle
1,60bca74dbea29a1eb66c8061,60bca74bea8bf89011f263d2,How would you describe your periods?,I donâ€™t have a period anymore
2,60c20f71bea29a1eb66c8a89,60c20f66eb1ab928fa08191a,How would you describe your periods?,I donâ€™t have a period anymore
3,60d26e40bea29a1eb66cabc3,60d26e3de535908d4d4b939a,How would you describe your periods?,Unpredictable
4,60c0afb2bea29a1eb66c8815,60c0afb1e019e61e41055177,How would you describe your periods?,Regular cycle
5,60df75e0bea29a1eb66cc52d,60df75de27df7d3103e0d33c,How would you describe your periods?,Unpredictable
6,60c1a154bea29a1eb66c89f6,60c1a151eb1ab928fa08189a,How would you describe your periods?,I donâ€™t have a period anymore
7,60c1c8abbea29a1eb66c8a0d,60c1c8a962779fa5ce932304,How would you describe your periods?,I donâ€™t have a period anymore
8,60b978c2bea29a1eb66c7ad8,60b978c063a11ed543ff5589,How would you describe your periods?,Unpredictable
9,6106f9d0bea29a1eb66d1dc3,6106f9ce078c683a7ea3f6c8,How would you describe your periods?,Regular cycle


In [ ]:
results.shape

(33512, 4)

In [ ]:
symptoms = {}
for ans in results['answers'].values:
    sym = ans.split(',')
    for s in sym:
        temp = s.strip()
        if not (temp in symptoms):
            symptoms[temp] = 0
        symptoms[temp] = symptoms[temp]+1

In [ ]:
symptoms

{'Regular cycle': 6420,
 'I donâ€™t have a period anymore': 12017,
 'Unpredictable': 13642,
 'I take birth control': 1433,
 'and donâ€™t have a period': 1433}

Now instead ask about symptoms

In [ ]:
query = """
select * from onboardingpp 
  where (questions = 'Let''s customize your personal health tracker!' 
  or questions = 'Choose the symptoms you deal with the most.')
  and length(answers) > 0
"""

results = db.execQuery(query, cached=True)

In [ ]:
results.head()

,healthid,community_id,questions,answers
0,62a22ea6bea29a1eb6723576,62a22ea4f57de32eb215b8cb,Choose the symptoms you deal with the most.,"Bloating,Brittle Hair/Nails,Hot Flashes/Night ..."
1,62a258c6bea29a1eb67235f4,62a258c3f57de32eb215b96b,Choose the symptoms you deal with the most.,"Brittle Hair/Nails,Difficulty Concentrating,Ho..."
2,62a33009bea29a1eb6723904,62a33006f57de32eb215bb4d,Choose the symptoms you deal with the most.,"Anxiety,Fatigue,Difficulty Concentrating,Hot F..."
3,62a628cabea29a1eb6724371,62a628c7607213a9ae6e8a75,Choose the symptoms you deal with the most.,"Bloating,Brittle Hair/Nails,Fatigue,Hot Flashe..."
4,62a69141bea29a1eb67244eb,62a6913e607213a9ae6e8bb6,Choose the symptoms you deal with the most.,"Anxiety,Bloating,Difficulty Concentrating,Fati..."


In [ ]:
results.shape

(41725, 4)

In [ ]:
symptoms = {}
for ans in results['answers'].values:
    sym = ans.split(',')
    for s in sym:
        temp = s.strip()
        if not (temp in symptoms):
            symptoms[temp] = 0
        symptoms[temp] = symptoms[temp]+1

In [ ]:
symptoms

{'Bloating': 23163,
 'Brittle Hair/Nails': 14391,
 'Hot Flashes/Night Sweats': 26051,
 'Fatigue': 28056,
 'Insomnia': 20500,
 'Low Sex Drive': 20495,
 'Mood Swings/Irritability': 24242,
 'Signs of Aging Skin': 17970,
 'Weight Gain': 29567,
 'Difficulty Concentrating': 20850,
 'Irregular Periods': 15175,
 'Anxiety': 22827,
 "I don't really have any symptoms that bother me.": 344,
 "I don't really have any symptoms that bother me": 226}

In [ ]:
symptoms['I don\'t really have any symptoms that bother me.'] += symptoms['I don\'t really have any symptoms that bother me']

In [ ]:
del symptoms['I don\'t really have any symptoms that bother me']

In [ ]:
# Create a frame from this, so we can save it as a nice little table. Might come handy in some situation.

frame = pd.DataFrame.from_dict(symptoms, orient='index', columns=['count'])
frame.sort_values(by=['count'], ascending=False)

,count
Weight Gain,29567
Fatigue,28056
Hot Flashes/Night Sweats,26051
Mood Swings/Irritability,24242
Bloating,23163
Anxiety,22827
Difficulty Concentrating,20850
Insomnia,20500
Low Sex Drive,20495
Signs of Aging Skin,17970


Network Analysis


In [ ]:
def createHotEncoding(symptoms, results):
    """
    Creates a one hot encoding for the symptoms, which can then be fed into a NA algorithm using R.
    
    symptoms : A list of all the symptoms we should include.
    results : The complete dataframe from onboarding that has the answers and userid.
    
    Returns a dataframe with the one hot encoding.
    """
    columns = symptoms

    data = pd.DataFrame(columns=columns)
    indexm = {}
    for i, v in enumerate(columns):
        indexm[v] = i

    row = np.zeros(len(columns), dtype='int64')
    row[2] = 1
    row[4] = 1
    series = pd.Series(row, index=columns)
    series = series.to_frame().T
    data = pd.concat([data, series], ignore_index=True)

    for (uid, ans) in zip(results['healthid'], results['answers']):
        sym = utils.extractSymptoms(ans)
        row = np.zeros(len(columns), dtype='int64')
        #row[-1] = uid # Fill the uid in at the end.
        for s in sym:
            s = s.strip()
            # This is not really a symptom so we don't care much about it.
            if s == "I don\'t really have any symptoms that bother me" or s == "I don\'t really have any symptoms that bother me.":
                continue
            row[indexm[s]] = 1
        series = pd.Series(row, index=columns)
        series = series.to_frame().T
        data = pd.concat([data, series], ignore_index=True)
        
    # Drop the column for "I don't really have any symptoms"
    data = data.drop(["I don't really have any symptoms that bother me."], axis=1)

    return data

In [ ]:
data = createHotEncoding(frame.index.values, results)